In [11]:
import requests
import os
import json
from datetime import datetime, timedelta
from collections import defaultdict
import time

# =================================================================
# ADD YOUR API KEYS BELOW
# =================================================================
# Amadeus Flight API
AMADEUS_API_KEY = "HURxCHKW5kWgbvGjoGUeYstlgM18YviN"  # Replace with your actual API key
AMADEUS_API_SECRET = "U8NoNpzurIlLkbXG"  # Replace with your actual API secret

# OpenWeatherMap API
WEATHER_API_KEY = "33d17bb8ef6aca6452a797dbf60fe4b1"  # Replace with your actual API key

# Booking.com RapidAPI
BOOKING_API_KEY = "e65d8ee95fmsh038b47ad4c8519ep1d34acjsn493d5bc0a51a"  # Replace with your actual API key
BOOKING_API_HOST = "booking-com15.p.rapidapi.com"  # Keep this as is or replace if different

# Airport code to city name mapping
AIRPORT_MAPPING = {
    "JFK": "New York",
    "CDG": "Paris",
    "LAX": "Los Angeles",
    "LHR": "London",
    "NRT": "Tokyo",
    "SYD": "Sydney",
    "DXB": "Dubai",
    "SIN": "Singapore",
    "HKG": "Hong Kong",
    "FCO": "Rome"
}
# =================================================================
# END OF API KEYS SECTION
# =================================================================

# Base Agent class with common functionality
class BaseAgent:
    """Base class for all agents in the travel assistant system"""

    def __init__(self, name):
        self.name = name
        self.data = None

    def process_request(self, *args, **kwargs):
        """Main method to be implemented by each agent"""
        raise NotImplementedError("Each agent must implement its own process_request method")

    def log_info(self, message):
        """Log information with agent identifier"""
        print(f"[{self.name}] {message}")

    def log_error(self, message):
        """Log errors with agent identifier"""
        print(f"[{self.name} ERROR] {message}")

    def get_data(self):
        """Return processed data"""
        return self.data


class FlightAgent(BaseAgent):
    """Agent responsible for retrieving flight information"""

    def __init__(self):
        super().__init__("Flight Agent")
        self.api_key = AMADEUS_API_KEY
        self.api_secret = AMADEUS_API_SECRET
        self.token = None
        self.token_expiry = None
        self.base_url = "https://test.api.amadeus.com"

    def _authenticate(self):
        """Get authentication token from Amadeus API"""
        # Check if token is still valid
        if self.token and self.token_expiry and datetime.now() < self.token_expiry:
            return True

        self.log_info("Authenticating with Amadeus API...")
        auth_url = f"{self.base_url}/v1/security/oauth2/token"
        auth_headers = {"Content-Type": "application/x-www-form-urlencoded"}
        auth_data = {
            "grant_type": "client_credentials",
            "client_id": self.api_key,
            "client_secret": self.api_secret
        }

        try:
            response = requests.post(auth_url, headers=auth_headers, data=auth_data)
            response.raise_for_status()
            auth_data = response.json()
            self.token = auth_data["access_token"]
            # Set token expiry time (usually 30 minutes)
            expires_in = auth_data.get("expires_in", 1800)  # Default 30 minutes
            self.token_expiry = datetime.now() + timedelta(seconds=expires_in)
            self.log_info("Authentication successful")
            return True
        except Exception as e:
            self.log_error(f"Authentication failed: {str(e)}")
            return False

    def process_request(self, origin, destination, departure_date, return_date=None, adults=1):
        """Fetch flight information from Amadeus API"""
        if not self._authenticate():
            self.log_error("Cannot process request without authentication")
            return None

        self.log_info(f"Searching flights from {origin} to {destination} on {departure_date}")

        # Prepare request to flight offers search endpoint
        flights_url = f"{self.base_url}/v2/shopping/flight-offers"
        headers = {"Authorization": f"Bearer {self.token}"}
        params = {
            "originLocationCode": origin,
            "destinationLocationCode": destination,
            "departureDate": departure_date,
            "adults": adults,
            "max": 5  # Limit results to 5 options
        }

        # Add return date if provided
        if return_date:
            params["returnDate"] = return_date

        try:
            response = requests.get(flights_url, headers=headers, params=params)
            response.raise_for_status()
            flight_data = response.json().get("data", [])

            # Process flight data into a more usable format
            processed_flights = []
            for flight in flight_data:
                # Extract basic information from each flight
                flight_info = {
                    "price": f"{flight['price']['total']} {flight['price']['currency']}",
                    "outbound": [],
                    "inbound": []
                }

                # Process outbound itinerary
                for segment in flight["itineraries"][0]["segments"]:
                    flight_info["outbound"].append({
                        "airline": segment.get("carrierCode", "Unknown"),
                        "flight_number": segment.get("number", "Unknown"),
                        "departure": {
                            "airport": segment["departure"]["iataCode"],
                            "time": segment["departure"]["at"]
                        },
                        "arrival": {
                            "airport": segment["arrival"]["iataCode"],
                            "time": segment["arrival"]["at"]
                        },
                        "duration": segment.get("duration", "Unknown")
                    })

                # Process inbound itinerary if it exists
                if len(flight["itineraries"]) > 1 and return_date:
                    for segment in flight["itineraries"][1]["segments"]:
                        flight_info["inbound"].append({
                            "airline": segment.get("carrierCode", "Unknown"),
                            "flight_number": segment.get("number", "Unknown"),
                            "departure": {
                                "airport": segment["departure"]["iataCode"],
                                "time": segment["departure"]["at"]
                            },
                            "arrival": {
                                "airport": segment["arrival"]["iataCode"],
                                "time": segment["arrival"]["at"]
                            },
                            "duration": segment.get("duration", "Unknown")
                        })

                processed_flights.append(flight_info)

            self.data = processed_flights
            self.log_info(f"Found {len(processed_flights)} flight options")
            return processed_flights
        except Exception as e:
            self.log_error(f"Failed to fetch flights: {str(e)}")
            if hasattr(e, 'response') and e.response:
                self.log_error(f"Response: {e.response.text}")
            return None


class WeatherAgent(BaseAgent):
    """Agent responsible for retrieving weather information"""

    def __init__(self):
        super().__init__("Weather Agent")
        self.api_key = WEATHER_API_KEY
        self.forecast_url = "https://api.openweathermap.org/data/2.5/forecast"

    def process_request(self, city, start_date, end_date):
        """Fetch weather forecast for a city during specified dates"""
        self.log_info(f"Retrieving weather forecast for {city} from {start_date} to {end_date}")

        # Convert string dates to datetime objects
        start_dt = datetime.strptime(start_date, "%Y-%m-%d")
        end_dt = datetime.strptime(end_date, "%Y-%m-%d")

        params = {
            "q": city,
            "appid": self.api_key,
            "units": "metric"  # Use metric units
        }

        try:
            response = requests.get(self.forecast_url, params=params)
            response.raise_for_status()
            weather_data = response.json()

            # Group forecast by date
            daily_forecasts = defaultdict(lambda: {"temps": [], "conditions": []})

            for forecast in weather_data.get("list", []):
                forecast_time = datetime.fromtimestamp(forecast["dt"])
                forecast_date = forecast_time.date()

                # Only include dates within our range
                if start_dt.date() <= forecast_date <= end_dt.date():
                    date_key = forecast_date.strftime("%Y-%m-%d")
                    daily_forecasts[date_key]["temps"].append(forecast["main"]["temp"])
                    daily_forecasts[date_key]["conditions"].append(forecast["weather"][0]["main"])
                    daily_forecasts[date_key]["description"] = forecast["weather"][0]["description"]
                    daily_forecasts[date_key]["humidity"] = forecast["main"]["humidity"]
                    daily_forecasts[date_key]["wind_speed"] = forecast["wind"]["speed"]

            # Create summary for each day
            weather_summary = {}
            for date, data in daily_forecasts.items():
                if data["temps"]:  # Ensure we have data for this day
                    # Find most common weather condition
                    condition_counts = {}
                    for condition in data["conditions"]:
                        condition_counts[condition] = condition_counts.get(condition, 0) + 1
                    most_common_condition = max(condition_counts.items(), key=lambda x: x[1])[0]

                    # Calculate average temperature
                    avg_temp = sum(data["temps"]) / len(data["temps"])

                    weather_summary[date] = {
                        "condition": most_common_condition,
                        "description": data["description"],
                        "average_temp": round(avg_temp, 1),
                        "humidity": data["humidity"],
                        "wind_speed": data["wind_speed"]
                    }

            self.data = weather_summary
            self.log_info(f"Retrieved weather data for {len(weather_summary)} days")
            return weather_summary
        except Exception as e:
            self.log_error(f"Failed to fetch weather data: {str(e)}")
            return None


class HotelAgent(BaseAgent):
    """Agent responsible for retrieving hotel information"""

    def __init__(self):
        super().__init__("Hotel Agent")
        self.api_key = BOOKING_API_KEY
        self.api_host = BOOKING_API_HOST
        self.base_url = "https://booking-com15.p.rapidapi.com/api/v1/hotels/searchHotelsByCoordinates"
        self.geocode_url = "http://api.openweathermap.org/geo/1.0/direct"
        self.weather_api_key = WEATHER_API_KEY

    def _get_coordinates(self, city):
        """Get geographic coordinates for a city using OpenWeatherMap API"""
        self.log_info(f"Getting coordinates for {city}")

        params = {
            "q": city,
            "limit": 1,
            "appid": self.weather_api_key
        }

        try:
            response = requests.get(self.geocode_url, params=params)
            response.raise_for_status()
            location_data = response.json()

            if location_data and len(location_data) > 0:
                lat = location_data[0]["lat"]
                lon = location_data[0]["lon"]
                self.log_info(f"Found coordinates: {lat}, {lon}")
                return lat, lon
            else:
                self.log_error(f"No coordinates found for {city}")
                return None, None
        except Exception as e:
            self.log_error(f"Error getting coordinates: {str(e)}")
            return None, None

    def process_request(self, city, check_in_date, check_out_date, adults=1, rooms=1, limit=5):
        """Find hotel accommodations in the specified city"""
        lat, lon = self._get_coordinates(city)

        if not lat or not lon:
            self.log_error("Cannot search for hotels without coordinates")
            return None

        self.log_info(f"Searching for hotels in {city} from {check_in_date} to {check_out_date}")

        headers = {
            "X-RapidAPI-Key": self.api_key,
            "X-RapidAPI-Host": self.api_host
        }

        params = {
            "latitude": lat,
            "longitude": lon,
            "arrival_date": check_in_date,
            "departure_date": check_out_date,
            "adults": adults,
            "room_qty": rooms,
            "page_number": 1,
            "languagecode": "en-us",
            "currency_code": "USD",
            "units": "metric",
            "limit": limit
        }

        try:
            response = requests.get(self.base_url, headers=headers, params=params)
            response.raise_for_status()
            hotels_data = response.json()

            # Extract hotel results
            hotels_list = hotels_data.get("data", {}).get("result", [])

            if not isinstance(hotels_list, list):
                self.log_error("Invalid response format from hotel API")
                return None

            # Process and format hotel information
            processed_hotels = []
            for hotel in hotels_list:
                hotel_info = {
                    "name": hotel.get("hotel_name", "Unknown Hotel"),
                    "price": f"{hotel.get('min_total_price', 'N/A')} {hotel.get('currencycode', 'USD')}",
                    "rating": hotel.get("review_score", "N/A"),
                    "address": hotel.get("address", "N/A"),
                    "distance_to_center": f"{hotel.get('distance_to_cc', 'N/A')} km",
                    "stars": hotel.get("hotel_class", "N/A")
                }
                processed_hotels.append(hotel_info)

            self.data = processed_hotels
            self.log_info(f"Found {len(processed_hotels)} hotel options")
            return processed_hotels
        except Exception as e:
            self.log_error(f"Failed to fetch hotels: {str(e)}")
            return None


class ItineraryPlannerAgent(BaseAgent):
    """Agent responsible for creating a comprehensive travel itinerary"""

    def __init__(self):
        super().__init__("Itinerary Planner")

    def process_request(self, city, start_date, end_date, flights, weather, hotels):
        """Generate a complete travel itinerary based on data from other agents"""
        self.log_info(f"Creating itinerary for trip to {city}")

        # Format dates for display
        start_dt = datetime.strptime(start_date, "%Y-%m-%d")
        end_dt = datetime.strptime(end_date, "%Y-%m-%d")
        date_range = f"{start_dt.strftime('%B %d, %Y')} to {end_dt.strftime('%B %d, %Y')}"

        # Create the itinerary document
        itinerary = f"TRAVEL ITINERARY: {city.upper()}\n"
        itinerary += f"Travel Period: {date_range}\n"
        itinerary += "=" * 50 + "\n\n"

        # Add flight information
        itinerary += "FLIGHT DETAILS\n"
        itinerary += "-" * 30 + "\n"

        if flights and len(flights) > 0:
            for i, flight in enumerate(flights[:3], 1):
                outbound = flight.get("outbound", [])
                inbound = flight.get("inbound", [])

                itinerary += f"Option {i} - Price: {flight.get('price', 'N/A')}\n"

                # Outbound journey
                if outbound:
                    itinerary += "  OUTBOUND:\n"
                    for j, segment in enumerate(outbound, 1):
                        itinerary += f"    Segment {j}: {segment.get('airline', '')} {segment.get('flight_number', '')}\n"
                        itinerary += f"    From: {segment.get('departure', {}).get('airport', '')} at {segment.get('departure', {}).get('time', '')}\n"
                        itinerary += f"    To: {segment.get('arrival', {}).get('airport', '')} at {segment.get('arrival', {}).get('time', '')}\n"
                        itinerary += f"    Duration: {segment.get('duration', '')}\n"

                # Inbound journey
                if inbound:
                    itinerary += "  RETURN:\n"
                    for j, segment in enumerate(inbound, 1):
                        itinerary += f"    Segment {j}: {segment.get('airline', '')} {segment.get('flight_number', '')}\n"
                        itinerary += f"    From: {segment.get('departure', {}).get('airport', '')} at {segment.get('departure', {}).get('time', '')}\n"
                        itinerary += f"    To: {segment.get('arrival', {}).get('airport', '')} at {segment.get('arrival', {}).get('time', '')}\n"
                        itinerary += f"    Duration: {segment.get('duration', '')}\n"

                itinerary += "\n"
        else:
            itinerary += "No flight information available.\n\n"

        # Add weather information
        itinerary += "WEATHER FORECAST\n"
        itinerary += "-" * 30 + "\n"

        if weather and len(weather) > 0:
            for date, forecast in sorted(weather.items()):
                formatted_date = datetime.strptime(date, "%Y-%m-%d").strftime("%A, %B %d")
                itinerary += f"{formatted_date}:\n"
                itinerary += f"  Condition: {forecast.get('condition', 'N/A')} - {forecast.get('description', '')}\n"
                itinerary += f"  Average Temperature: {forecast.get('average_temp', 'N/A')}°C\n"
                itinerary += f"  Humidity: {forecast.get('humidity', 'N/A')}%\n"
                itinerary += f"  Wind Speed: {forecast.get('wind_speed', 'N/A')} m/s\n\n"
        else:
            itinerary += "No weather information available.\n\n"

        # Add accommodation options
        itinerary += "ACCOMMODATION OPTIONS\n"
        itinerary += "-" * 30 + "\n"

        if hotels and len(hotels) > 0:
            for i, hotel in enumerate(hotels, 1):
                itinerary += f"Option {i}: {hotel.get('name', 'N/A')}\n"
                itinerary += f"  Rating: {hotel.get('rating', 'N/A')} / 10\n"
                itinerary += f"  Price: {hotel.get('price', 'N/A')}\n"
                itinerary += f"  Address: {hotel.get('address', 'N/A')}\n"
                itinerary += f"  Distance to Center: {hotel.get('distance_to_center', 'N/A')}\n"
                itinerary += f"  Stars: {hotel.get('stars', 'N/A')}\n\n"
        else:
            itinerary += "No accommodation information available.\n\n"

        # Add travel tips based on weather
        itinerary += "TRAVEL RECOMMENDATIONS\n"
        itinerary += "-" * 30 + "\n"

        # Check for rain in weather data
        rainy_days = 0
        avg_temp = 0
        has_weather_data = False

        if weather and len(weather) > 0:
            has_weather_data = True
            day_count = len(weather)
            for _, forecast in weather.items():
                if 'rain' in forecast.get('condition', '').lower():
                    rainy_days += 1
                avg_temp += forecast.get('average_temp', 0)

            if day_count > 0:
                avg_temp /= day_count

        if has_weather_data:
            if rainy_days > 0:
                itinerary += "• Pack an umbrella or raincoat as rain is expected during your trip.\n"

            if avg_temp < 10:
                itinerary += "• Weather will be cold - pack warm clothing layers.\n"
            elif avg_temp < 20:
                itinerary += "• Weather will be mild - pack light layers for comfort.\n"
            else:
                itinerary += "• Weather will be warm - pack light, breathable clothing.\n"

        itinerary += "• Remember to check-in online 24 hours before your flight.\n"
        itinerary += "• Carry a printed copy of your hotel booking confirmation.\n"
        itinerary += "• Exchange some currency beforehand for immediate expenses upon arrival.\n"
        itinerary += "• Keep important documents (passport, insurance, etc.) in a secure place.\n\n"

        # Closing
        itinerary += "=" * 50 + "\n"
        itinerary += "Itinerary generated on " + datetime.now().strftime("%Y-%m-%d at %H:%M:%S") + "\n"
        itinerary += "Have a wonderful trip to " + city + "!\n"

        self.data = itinerary
        return itinerary


class TravelAssistantOrchestrator:
    """Main class that coordinates all the travel agents"""

    def __init__(self):
        self.flight_agent = FlightAgent()
        self.weather_agent = WeatherAgent()
        self.hotel_agent = HotelAgent()
        self.itinerary_agent = ItineraryPlannerAgent()

    def get_city_from_airport(self, airport_code):
        """Convert airport code to city name"""
        return AIRPORT_MAPPING.get(airport_code, airport_code)

    def plan_trip(self, origin, destination, start_date, end_date, adults=1):
        """Generate a complete travel itinerary"""
        print(f"\n{'='*20} TRAVEL ASSISTANT {'='*20}")
        print(f"Planning trip from {origin} to {destination}")
        print(f"Travel dates: {start_date} to {end_date}")
        print(f"Number of travelers: {adults}")
        print(f"{'='*60}\n")

        # Step 1: Get flight information
        print("Step 1: Retrieving flight options...")
        flights = self.flight_agent.process_request(origin, destination, start_date, end_date, adults)

        # Step 2: Get destination city and weather information
        destination_city = self.get_city_from_airport(destination)
        print(f"Step 2: Retrieving weather forecast for {destination_city}...")
        weather = self.weather_agent.process_request(destination_city, start_date, end_date)

        # Step 3: Get hotel options
        print(f"Step 3: Finding accommodation in {destination_city}...")
        hotels = self.hotel_agent.process_request(destination_city, start_date, end_date, adults)

        # Step 4: Generate itinerary
        print("Step 4: Creating your personalized travel itinerary...")
        itinerary = self.itinerary_agent.process_request(
            destination_city, start_date, end_date, flights, weather, hotels
        )

        print("\nYour travel itinerary is ready!")
        return itinerary

    def interactive_planning(self):
        """Interactive mode for planning a trip with user input"""
        print("\nWelcome to the AI Travel Assistant!\n")

        # Get origin
        origin = input("Please enter your departure airport code (e.g., JFK): ").strip().upper()

        # Get destination
        destination = input("Please enter your destination airport code (e.g., CDG): ").strip().upper()

        # Get travel dates
        start_date = input("Please enter your departure date (YYYY-MM-DD): ").strip()
        end_date = input("Please enter your return date (YYYY-MM-DD): ").strip()

        # Get number of travelers
        try:
            adults = int(input("Please enter the number of travelers: ").strip())
        except ValueError:
            print("Using default value of 1 traveler.")
            adults = 1

        # Generate and print the itinerary
        itinerary = self.plan_trip(origin, destination, start_date, end_date, adults)
        print("\n" + itinerary)

        return itinerary


# Main execution block
if __name__ == "__main__":
    # Create travel assistant
    travel_assistant = TravelAssistantOrchestrator()

    # Option 1: Direct method call with hard-coded values
    # itinerary = travel_assistant.plan_trip("JFK", "CDG", "2025-05-15", "2025-05-20", 2)
    # print(itinerary)

    # Option 2: Interactive planning (recommended)
    travel_assistant.interactive_planning()


Welcome to the AI Travel Assistant!

Please enter your departure airport code (e.g., JFK): JFK
Please enter your destination airport code (e.g., CDG): CDG
Please enter your departure date (YYYY-MM-DD): 2025-04-22
Please enter your return date (YYYY-MM-DD): 2025-04-24
Please enter the number of travelers: 4

==================== TRAVEL ASSISTANT ====================
Planning trip from JFK to CDG
Travel dates: 2025-04-22 to 2025-04-24
Number of travelers: 4

Step 1: Retrieving flight options...
[Flight Agent] Authenticating with Amadeus API...
[Flight Agent] Authentication successful
[Flight Agent] Searching flights from JFK to CDG on 2025-04-22
[Flight Agent] Found 5 flight options
Step 2: Retrieving weather forecast for Paris...
[Weather Agent] Retrieving weather forecast for Paris from 2025-04-22 to 2025-04-24
[Weather Agent] Retrieved weather data for 3 days
Step 3: Finding accommodation in Paris...
[Hotel Agent] Getting coordinates for Paris
[Hotel Agent] Found coordinates: 48.8588

In [13]:

# Main execution block
if __name__ == "__main__":
    # Create travel assistant
    travel_assistant = TravelAssistantOrchestrator()

    # Option 1: Direct method call with hard-coded values
    # itinerary = travel_assistant.plan_trip("JFK", "CDG", "2025-05-15", "2025-05-20", 2)
    # print(itinerary)

    # Option 2: Interactive planning (recommended)
    travel_assistant.interactive_planning()


Welcome to the AI Travel Assistant!

Please enter your departure airport code (e.g., JFK): LAX
Please enter your destination airport code (e.g., CDG): SYD
Please enter your departure date (YYYY-MM-DD): 2025-04-22
Please enter your return date (YYYY-MM-DD): 2025-04-24
Please enter the number of travelers: 2

==================== TRAVEL ASSISTANT ====================
Planning trip from LAX to SYD
Travel dates: 2025-04-22 to 2025-04-24
Number of travelers: 2

Step 1: Retrieving flight options...
[Flight Agent] Authenticating with Amadeus API...
[Flight Agent] Authentication successful
[Flight Agent] Searching flights from LAX to SYD on 2025-04-22
[Flight Agent] Found 5 flight options
Step 2: Retrieving weather forecast for Sydney...
[Weather Agent] Retrieving weather forecast for Sydney from 2025-04-22 to 2025-04-24
[Weather Agent] Retrieved weather data for 3 days
Step 3: Finding accommodation in Sydney...
[Hotel Agent] Getting coordinates for Sydney
[Hotel Agent] Found coordinates: -33

In [14]:

# Main execution block
if __name__ == "__main__":
    # Create travel assistant
    travel_assistant = TravelAssistantOrchestrator()

    # Option 1: Direct method call with hard-coded values
    # itinerary = travel_assistant.plan_trip("JFK", "CDG", "2025-05-15", "2025-05-20", 2)
    # print(itinerary)

    # Option 2: Interactive planning (recommended)
    travel_assistant.interactive_planning()


Welcome to the AI Travel Assistant!

Please enter your departure airport code (e.g., JFK): DXB
Please enter your destination airport code (e.g., CDG): HKG
Please enter your departure date (YYYY-MM-DD): 2025-04-22
Please enter your return date (YYYY-MM-DD): 2025-04-24
Please enter the number of travelers: 1

==================== TRAVEL ASSISTANT ====================
Planning trip from DXB to HKG
Travel dates: 2025-04-22 to 2025-04-24
Number of travelers: 1

Step 1: Retrieving flight options...
[Flight Agent] Authenticating with Amadeus API...
[Flight Agent] Authentication successful
[Flight Agent] Searching flights from DXB to HKG on 2025-04-22
[Flight Agent] Found 5 flight options
Step 2: Retrieving weather forecast for Hong Kong...
[Weather Agent] Retrieving weather forecast for Hong Kong from 2025-04-22 to 2025-04-24
[Weather Agent] Retrieved weather data for 3 days
Step 3: Finding accommodation in Hong Kong...
[Hotel Agent] Getting coordinates for Hong Kong
[Hotel Agent] Found coor

SECOND ONE


In [1]:
import requests
import os
import json
from datetime import datetime, timedelta
from collections import defaultdict
import time

# =================================================================
# ADD YOUR API KEYS BELOW
# =================================================================
# Amadeus Flight API
AMADEUS_API_KEY = "HURxCHKW5kWgbvGjoGUeYstlgM18YviN"  # Replace with your actual API key
AMADEUS_API_SECRET = "U8NoNpzurIlLkbXG"  # Replace with your actual API secret
# OpenWeatherMap API
WEATHER_API_KEY = "33d17bb8ef6aca6452a797dbf60fe4b1"  # Replace with your actual API key

# Booking.com RapidAPI
BOOKING_API_KEY = "e65d8ee95fmsh038b47ad4c8519ep1d34acjsn493d5bc0a51a"  # Replace with your actual API key
BOOKING_API_HOST = "booking-com15.p.rapidapi.com"  # Keep this as is or replace if different
# Airport code to city name mapping
AIRPORT_MAPPING = {
    "JFK": "New York",
    "CDG": "Paris",
    "LAX": "Los Angeles",
    "LHR": "London",
    "NRT": "Tokyo",
    "SYD": "Sydney",
    "DXB": "Dubai",
    "SIN": "Singapore",
    "HKG": "Hong Kong",
    "FCO": "Rome"
}

# Airline code to name mapping
AIRLINE_MAPPING = {
    "6X": "Amadeus Test Airline",
    "AF": "Air France",
    "BA": "British Airways",
    "LH": "Lufthansa",
    "DL": "Delta Air Lines",
    "UA": "United Airlines",
    "AA": "American Airlines",
    "EK": "Emirates",
    "QR": "Qatar Airways",
    "SQ": "Singapore Airlines",
    "CX": "Cathay Pacific",
    "JL": "Japan Airlines"
}
# =================================================================
# END OF API KEYS SECTION
# =================================================================

# Base Agent class with common functionality
class BaseAgent:
    """Base class for all agents in the travel assistant system"""

    def __init__(self, name):
        self.name = name
        self.data = None

    def process_request(self, *args, **kwargs):
        """Main method to be implemented by each agent"""
        raise NotImplementedError("Each agent must implement its own process_request method")

    def log_info(self, message):
        """Log information with agent identifier"""
        print(f"[{self.name}] {message}")

    def log_error(self, message):
        """Log errors with agent identifier"""
        print(f"[{self.name} ERROR] {message}")

    def get_data(self):
        """Return processed data"""
        return self.data


class FlightAgent(BaseAgent):
    """Agent responsible for retrieving flight information"""

    def __init__(self):
        super().__init__("Flight Agent")
        self.api_key = AMADEUS_API_KEY
        self.api_secret = AMADEUS_API_SECRET
        self.token = None
        self.token_expiry = None
        self.base_url = "https://test.api.amadeus.com"
        self.airline_mapping = AIRLINE_MAPPING

    def _authenticate(self):
        """Get authentication token from Amadeus API"""
        # Check if token is still valid
        if self.token and self.token_expiry and datetime.now() < self.token_expiry:
            return True

        self.log_info("Authenticating with Amadeus API...")
        auth_url = f"{self.base_url}/v1/security/oauth2/token"
        auth_headers = {"Content-Type": "application/x-www-form-urlencoded"}
        auth_data = {
            "grant_type": "client_credentials",
            "client_id": self.api_key,
            "client_secret": self.api_secret
        }

        try:
            response = requests.post(auth_url, headers=auth_headers, data=auth_data)
            response.raise_for_status()
            auth_data = response.json()
            self.token = auth_data["access_token"]
            # Set token expiry time (usually 30 minutes)
            expires_in = auth_data.get("expires_in", 1800)  # Default 30 minutes
            self.token_expiry = datetime.now() + timedelta(seconds=expires_in)
            self.log_info("Authentication successful")
            return True
        except Exception as e:
            self.log_error(f"Authentication failed: {str(e)}")
            return False

    def _get_airline_name(self, airline_code):
        """Get the full airline name from the airline code"""
        return self.airline_mapping.get(airline_code, airline_code)

    def _format_duration(self, duration_str):
        """Format duration string to be more readable"""
        # PT4H30M -> 4h 30m
        if not duration_str.startswith("PT"):
            return duration_str

        duration = duration_str[2:]  # Remove PT prefix

        hours = 0
        minutes = 0

        if "H" in duration:
            hours_parts = duration.split("H")
            hours = int(hours_parts[0])
            duration = hours_parts[1]

        if "M" in duration:
            minutes_parts = duration.split("M")
            minutes = int(minutes_parts[0])

        if hours and minutes:
            return f"{hours}h {minutes}m"
        elif hours:
            return f"{hours}h"
        elif minutes:
            return f"{minutes}m"
        else:
            return duration_str

    def process_request(self, origin, destination, departure_date, return_date=None, adults=1):
        """Fetch flight information from Amadeus API"""
        if not self._authenticate():
            self.log_error("Cannot process request without authentication")
            return None

        self.log_info(f"Searching flights from {origin} to {destination} on {departure_date}")

        # Prepare request to flight offers search endpoint
        flights_url = f"{self.base_url}/v2/shopping/flight-offers"
        headers = {"Authorization": f"Bearer {self.token}"}
        params = {
            "originLocationCode": origin,
            "destinationLocationCode": destination,
            "departureDate": departure_date,
            "adults": adults,
            "max": 5  # Limit results to 5 options
        }

        # Add return date if provided
        if return_date:
            params["returnDate"] = return_date

        try:
            response = requests.get(flights_url, headers=headers, params=params)
            response.raise_for_status()
            flight_data = response.json().get("data", [])

            # Process flight data into a more usable format
            processed_flights = []
            for flight in flight_data:
                # Extract basic information from each flight
                flight_info = {
                    "price": f"{flight['price']['total']} {flight['price']['currency']}",
                    "outbound": [],
                    "inbound": []
                }

                # Process outbound itinerary
                for segment in flight["itineraries"][0]["segments"]:
                    airline_code = segment.get("carrierCode", "Unknown")
                    airline_name = self._get_airline_name(airline_code)
                    flight_number = segment.get("number", "Unknown")

                    flight_info["outbound"].append({
                        "airline": airline_code,
                        "airline_name": airline_name,
                        "flight_number": flight_number,
                        "departure": {
                            "airport": segment["departure"]["iataCode"],
                            "time": segment["departure"]["at"]
                        },
                        "arrival": {
                            "airport": segment["arrival"]["iataCode"],
                            "time": segment["arrival"]["at"]
                        },
                        "duration": self._format_duration(segment.get("duration", "Unknown"))
                    })

                # Process inbound itinerary if it exists
                if len(flight["itineraries"]) > 1 and return_date:
                    for segment in flight["itineraries"][1]["segments"]:
                        airline_code = segment.get("carrierCode", "Unknown")
                        airline_name = self._get_airline_name(airline_code)
                        flight_number = segment.get("number", "Unknown")

                        flight_info["inbound"].append({
                            "airline": airline_code,
                            "airline_name": airline_name,
                            "flight_number": flight_number,
                            "departure": {
                                "airport": segment["departure"]["iataCode"],
                                "time": segment["departure"]["at"]
                            },
                            "arrival": {
                                "airport": segment["arrival"]["iataCode"],
                                "time": segment["arrival"]["at"]
                            },
                            "duration": self._format_duration(segment.get("duration", "Unknown"))
                        })

                processed_flights.append(flight_info)

            self.data = processed_flights
            self.log_info(f"Found {len(processed_flights)} flight options")
            return processed_flights
        except Exception as e:
            self.log_error(f"Failed to fetch flights: {str(e)}")
            if hasattr(e, 'response') and e.response:
                self.log_error(f"Response: {e.response.text}")
            return None


class WeatherAgent(BaseAgent):
    """Agent responsible for retrieving weather information"""

    def __init__(self):
        super().__init__("Weather Agent")
        self.api_key = WEATHER_API_KEY
        self.forecast_url = "https://api.openweathermap.org/data/2.5/forecast"
        self.condition_explanations = {
            "Clear": "Clear skies with abundant sunshine",
            "Clouds": "Partly to mostly cloudy",
            "Rain": "Precipitation in the form of raindrops",
            "Drizzle": "Light rain consisting of tiny water droplets",
            "Thunderstorm": "Storm with lightning and thunder",
            "Snow": "Precipitation in the form of ice crystals",
            "Mist": "Thin fog with reduced visibility",
            "Fog": "Thick cloud at ground level with very reduced visibility"
        }

    def process_request(self, city, start_date, end_date):
        """Fetch weather forecast for a city during specified dates"""
        self.log_info(f"Retrieving weather forecast for {city} from {start_date} to {end_date}")

        # Convert string dates to datetime objects
        start_dt = datetime.strptime(start_date, "%Y-%m-%d")
        end_dt = datetime.strptime(end_date, "%Y-%m-%d")

        params = {
            "q": city,
            "appid": self.api_key,
            "units": "metric"  # Use metric units
        }

        try:
            response = requests.get(self.forecast_url, params=params)
            response.raise_for_status()
            weather_data = response.json()

            # Group forecast by date
            daily_forecasts = defaultdict(lambda: {"temps": [], "conditions": [], "humidity": [], "wind_speed": [], "descriptions": []})

            for forecast in weather_data.get("list", []):
                forecast_time = datetime.fromtimestamp(forecast["dt"])
                forecast_date = forecast_time.date()

                # Only include dates within our range
                if start_dt.date() <= forecast_date <= end_dt.date():
                    date_key = forecast_date.strftime("%Y-%m-%d")
                    daily_forecasts[date_key]["temps"].append(forecast["main"]["temp"])
                    daily_forecasts[date_key]["conditions"].append(forecast["weather"][0]["main"])
                    daily_forecasts[date_key]["descriptions"].append(forecast["weather"][0]["description"])
                    daily_forecasts[date_key]["humidity"].append(forecast["main"]["humidity"])
                    daily_forecasts[date_key]["wind_speed"].append(forecast["wind"]["speed"])

            # Create summary for each day
            weather_summary = {}
            for date, data in daily_forecasts.items():
                if data["temps"]:  # Ensure we have data for this day
                    # Find most common weather condition
                    condition_counts = {}
                    for condition in data["conditions"]:
                        condition_counts[condition] = condition_counts.get(condition, 0) + 1
                    most_common_condition = max(condition_counts.items(), key=lambda x: x[1])[0]

                    # Find most common description
                    description_counts = {}
                    for description in data["descriptions"]:
                        description_counts[description] = description_counts.get(description, 0) + 1
                    most_common_description = max(description_counts.items(), key=lambda x: x[1])[0]

                    # Calculate averages
                    avg_temp = sum(data["temps"]) / len(data["temps"])
                    avg_humidity = sum(data["humidity"]) / len(data["humidity"])
                    avg_wind_speed = sum(data["wind_speed"]) / len(data["wind_speed"])

                    # Get extended explanation for condition
                    condition_explanation = self.condition_explanations.get(most_common_condition, "")

                    # Calculate comfort level
                    comfort = self._get_comfort_level(avg_temp, avg_humidity, avg_wind_speed)

                    weather_summary[date] = {
                        "condition": most_common_condition,
                        "description": most_common_description,
                        "explanation": condition_explanation,
                        "average_temp": round(avg_temp, 1),
                        "min_temp": round(min(data["temps"]), 1),
                        "max_temp": round(max(data["temps"]), 1),
                        "humidity": round(avg_humidity),
                        "wind_speed": round(avg_wind_speed, 1),
                        "comfort": comfort
                    }

            self.data = weather_summary
            self.log_info(f"Retrieved weather data for {len(weather_summary)} days")
            return weather_summary
        except Exception as e:
            self.log_error(f"Failed to fetch weather data: {str(e)}")
            return None

    def _get_comfort_level(self, temp, humidity, wind_speed):
        """Calculate comfort level based on temperature, humidity and wind speed"""
        if temp < 0:
            return "Very Cold"
        elif temp < 10:
            return "Cold"
        elif temp < 15:
            return "Cool"
        elif temp < 25:
            if humidity > 70:
                return "Warm and Humid"
            else:
                return "Pleasant"
        elif temp < 30:
            if humidity > 70:
                return "Hot and Humid"
            else:
                return "Warm"
        else:
            if humidity > 70:
                return "Very Hot and Humid"
            else:
                return "Very Hot"


class HotelAgent(BaseAgent):
    """Agent responsible for retrieving hotel information"""

    def __init__(self):
        super().__init__("Hotel Agent")
        self.api_key = BOOKING_API_KEY
        self.api_host = BOOKING_API_HOST
        self.base_url = "https://booking-com15.p.rapidapi.com/api/v1/hotels/searchHotelsByCoordinates"
        self.geocode_url = "http://api.openweathermap.org/geo/1.0/direct"
        self.weather_api_key = WEATHER_API_KEY
        self.star_explanations = {
            "1": "Budget accommodation with basic amenities",
            "2": "Value accommodation with better amenities than 1-star",
            "3": "Mid-range accommodation with good amenities and services",
            "4": "Upscale accommodation with excellent amenities and services",
            "5": "Luxury accommodation with superior amenities and personalized services"
        }

    def _get_coordinates(self, city):
        """Get geographic coordinates for a city using OpenWeatherMap API"""
        self.log_info(f"Getting coordinates for {city}")

        params = {
            "q": city,
            "limit": 1,
            "appid": self.weather_api_key
        }

        try:
            response = requests.get(self.geocode_url, params=params)
            response.raise_for_status()
            location_data = response.json()

            if location_data and len(location_data) > 0:
                lat = location_data[0]["lat"]
                lon = location_data[0]["lon"]
                self.log_info(f"Found coordinates: {lat}, {lon}")
                return lat, lon
            else:
                self.log_error(f"No coordinates found for {city}")
                return None, None
        except Exception as e:
            self.log_error(f"Error getting coordinates: {str(e)}")
            return None, None

    def _get_star_rating_description(self, stars):
        """Get a description for a hotel star rating"""
        if not stars or stars == "N/A":
            return "Rating not available"

        try:
            star_int = int(float(stars))
            return self.star_explanations.get(str(star_int), "Custom rating")
        except (ValueError, TypeError):
            return "Custom rating"

    def process_request(self, city, check_in_date, check_out_date, adults=1, rooms=1, limit=10):
        """Find hotel accommodations in the specified city"""
        lat, lon = self._get_coordinates(city)

        if not lat or not lon:
            self.log_error("Cannot search for hotels without coordinates")
            return None

        self.log_info(f"Searching for hotels in {city} from {check_in_date} to {check_out_date}")

        headers = {
            "X-RapidAPI-Key": self.api_key,
            "X-RapidAPI-Host": self.api_host
        }

        params = {
            "latitude": lat,
            "longitude": lon,
            "arrival_date": check_in_date,
            "departure_date": check_out_date,
            "adults": adults,
            "room_qty": rooms,
            "page_number": 1,
            "languagecode": "en-us",
            "currency_code": "EUR",  # Using EUR as default
            "units": "metric",
            "limit": limit
        }

        try:
            response = requests.get(self.base_url, headers=headers, params=params)
            response.raise_for_status()
            hotels_data = response.json()

            # Extract hotel results
            hotels_list = hotels_data.get("data", {}).get("result", [])

            if not isinstance(hotels_list, list):
                self.log_error("Invalid response format from hotel API")
                return None

            # Process and format hotel information
            processed_hotels = []
            for hotel in hotels_list:
                # Extract various details with fallbacks to default values
                hotel_name = hotel.get("hotel_name", "Unknown Hotel")
                price = hotel.get("min_total_price", "N/A")
                currency = hotel.get("currencycode", "EUR")
                rating = hotel.get("review_score", "N/A")
                address = hotel.get("address", f"{hotel_name}, {city}")
                stars = hotel.get("hotel_class", "N/A")
                distance = hotel.get("distance_to_cc", "N/A")
                amenities = hotel.get("amenities", [])

                # Format amenities as a list
                amenities_list = []
                if isinstance(amenities, str):
                    amenities_list = [item.strip() for item in amenities.split(",")]
                elif isinstance(amenities, list):
                    amenities_list = amenities

                # Enhanced hotel details with better fallbacks
                hotel_info = {
                    "name": hotel_name,
                    "price": f"{price} {currency}" if price != "N/A" else "Price information not available",
                    "rating": rating if rating != "N/A" else "Rating not available",
                    "address": address if address and address != "N/A" else f"{hotel_name}, {city}",
                    "distance_to_center": f"{distance} km" if distance != "N/A" else "Distance information not available",
                    "stars": stars if stars != "N/A" else "Star rating not available",
                    "star_description": self._get_star_rating_description(stars),
                    "amenities": amenities_list[:5] if amenities_list else ["Amenities information not available"]
                }
                processed_hotels.append(hotel_info)

            self.data = processed_hotels
            self.log_info(f"Found {len(processed_hotels)} hotel options")
            return processed_hotels
        except Exception as e:
            self.log_error(f"Failed to fetch hotels: {str(e)}")
            return None


class ItineraryPlannerAgent(BaseAgent):
    """Agent responsible for creating a comprehensive travel itinerary"""

    def __init__(self):
        super().__init__("Itinerary Planner")

    def process_request(self, city, start_date, end_date, flights, weather, hotels):
        """Generate a complete travel itinerary based on data from other agents"""
        self.log_info(f"Creating itinerary for trip to {city}")

        # Format dates for display
        start_dt = datetime.strptime(start_date, "%Y-%m-%d")
        end_dt = datetime.strptime(end_date, "%Y-%m-%d")
        date_range = f"{start_dt.strftime('%B %d, %Y')} to {end_dt.strftime('%B %d, %Y')}"

        # Create the itinerary document
        itinerary = f"TRAVEL ITINERARY: {city.upper()}\n"
        itinerary += f"Travel Period: {date_range}\n"
        itinerary += "=" * 50 + "\n\n"

        # Add flight information
        itinerary += "FLIGHT DETAILS\n"
        itinerary += "-" * 30 + "\n"

        if flights and len(flights) > 0:
            for i, flight in enumerate(flights[:3], 1):  # Limit to top 3 options
                outbound = flight.get("outbound", [])
                inbound = flight.get("inbound", [])

                itinerary += f"Option {i} - Price: {flight.get('price', 'N/A')}\n"

                # Outbound journey
                if outbound:
                    itinerary += "  OUTBOUND:\n"
                    for j, segment in enumerate(outbound, 1):
                        airline = segment.get('airline', '')
                        airline_name = segment.get('airline_name', airline)
                        flight_number = segment.get('flight_number', '')

                        itinerary += f"    Segment {j}: {airline_name} {airline} {flight_number}\n"
                        itinerary += f"    From: {segment.get('departure', {}).get('airport', '')} at {segment.get('departure', {}).get('time', '')}\n"
                        itinerary += f"    To: {segment.get('arrival', {}).get('airport', '')} at {segment.get('arrival', {}).get('time', '')}\n"
                        itinerary += f"    Duration: {segment.get('duration', '')}\n"

                # Inbound journey
                if inbound:
                    itinerary += "  RETURN:\n"
                    for j, segment in enumerate(inbound, 1):
                        airline = segment.get('airline', '')
                        airline_name = segment.get('airline_name', airline)
                        flight_number = segment.get('flight_number', '')

                        itinerary += f"    Segment {j}: {airline_name} {airline} {flight_number}\n"
                        itinerary += f"    From: {segment.get('departure', {}).get('airport', '')} at {segment.get('departure', {}).get('time', '')}\n"
                        itinerary += f"    To: {segment.get('arrival', {}).get('airport', '')} at {segment.get('arrival', {}).get('time', '')}\n"
                        itinerary += f"    Duration: {segment.get('duration', '')}\n"

                itinerary += "\n"
        else:
            itinerary += "No flight information available.\n\n"

        # Add weather information
        itinerary += "WEATHER FORECAST\n"
        itinerary += "-" * 30 + "\n"

        if weather and len(weather) > 0:
            for date, forecast in sorted(weather.items()):
                formatted_date = datetime.strptime(date, "%Y-%m-%d").strftime("%A, %B %d")
                itinerary += f"{formatted_date}:\n"
                itinerary += f"  Condition: {forecast.get('condition', 'N/A')} - {forecast.get('description', '')}\n"
                itinerary += f"  Average Temperature: {forecast.get('average_temp', 'N/A')}°C\n"
                itinerary += f"  Humidity: {forecast.get('humidity', 'N/A')}%\n"
                itinerary += f"  Wind Speed: {forecast.get('wind_speed', 'N/A')} m/s\n"
                itinerary += f"  Comfort: {forecast.get('comfort', 'N/A')}\n\n"
        else:
            itinerary += "No weather information available.\n\n"

        # Add accommodation options
        itinerary += "ACCOMMODATION OPTIONS\n"
        itinerary += "-" * 30 + "\n"

        if hotels and len(hotels) > 0:
            for i, hotel in enumerate(hotels, 1):
                itinerary += f"Option {i}: {hotel.get('name', 'N/A')}\n"
                itinerary += f"  Rating: {hotel.get('rating', 'N/A')} / 10\n"
                itinerary += f"  Price: {hotel.get('price', 'N/A')}\n"
                itinerary += f"  Address: {hotel.get('address', 'N/A')}\n"
                itinerary += f"  Distance to Center: {hotel.get('distance_to_center', 'N/A')}\n"
                itinerary += f"  Stars: {hotel.get('stars', 'N/A')}"

                # Add star description if available
                star_description = hotel.get('star_description', '')
                if star_description and star_description != "Rating not available":
                    itinerary += f" - {star_description}"

                itinerary += "\n"

                # Add amenities if available
                amenities = hotel.get('amenities', [])
                if amenities and amenities[0] != "Amenities information not available":
                    itinerary += f"  Amenities: {', '.join(amenities)}\n"

                itinerary += "\n"
        else:
            itinerary += "No accommodation information available.\n\n"

        # Add travel recommendations based on weather
        itinerary += "TRAVEL RECOMMENDATIONS\n"
        itinerary += "-" * 30 + "\n"

        # Check for rain in weather data
        rainy_days = 0
        avg_temp = 0
        has_weather_data = False

        if weather and len(weather) > 0:
            has_weather_data = True
            day_count = len(weather)
            for _, forecast in weather.items():
                if 'rain' in forecast.get('condition', '').lower():
                    rainy_days += 1
                avg_temp += forecast.get('average_temp', 0)

            if day_count > 0:
                avg_temp /= day_count

        if has_weather_data:
            if rainy_days > 0:
                itinerary += "• Pack an umbrella or raincoat as rain is expected during your trip.\n"

            if avg_temp < 10:
                itinerary += "• Weather will be cold - pack warm clothing layers.\n"
            elif avg_temp < 20:
                itinerary += "• Weather will be mild - pack light layers for comfort.\n"
            else:
                itinerary += "• Weather will be warm - pack light, breathable clothing.\n"

        itinerary += "• Remember to check-in online 24 hours before your flight.\n"
        itinerary += "• Carry a printed copy of your hotel booking confirmation.\n"
        itinerary += "• Exchange some currency beforehand for immediate expenses upon arrival.\n"
        itinerary += "• Keep important documents (passport, insurance, etc.) in a secure place.\n\n"

        # Closing
        itinerary += "=" * 50 + "\n"
        itinerary += "Itinerary generated on " + datetime.now().strftime("%Y-%m-%d at %H:%M:%S") + "\n"
        itinerary += f"Have a wonderful trip to {city}!\n"

        self.data = itinerary
        return itinerary


class TravelAssistantOrchestrator:
    """Main class that coordinates all the travel agents"""

    def __init__(self):
        self.flight_agent = FlightAgent()
        self.weather_agent = WeatherAgent()
        self.hotel_agent = HotelAgent()
        self.itinerary_agent = ItineraryPlannerAgent()

    def get_city_from_airport(self, airport_code):
        """Convert airport code to city name"""
        return AIRPORT_MAPPING.get(airport_code, airport_code)

    def plan_trip(self, origin, destination, start_date, end_date, adults=1):
        """Generate a complete travel itinerary"""
        print(f"\n{'='*20} TRAVEL ASSISTANT {'='*20}")
        print(f"Planning trip from {origin} to {destination}")
        print(f"Travel dates: {start_date} to {end_date}")
        print(f"Number of travelers: {adults}")
        print(f"{'='*60}\n")

        # Step 1: Get flight information
        print("Step 1: Retrieving flight options...")
        flights = self.flight_agent.process_request(origin, destination, start_date, end_date, adults)

        # Step 2: Get destination city and weather information
        destination_city = self.get_city_from_airport(destination)
        print(f"Step 2: Retrieving weather forecast for {destination_city}...")
        weather = self.weather_agent.process_request(destination_city, start_date, end_date)

        # Step 3: Get hotel options
        print(f"Step 3: Finding accommodation in {destination_city}...")
        hotels = self.hotel_agent.process_request(destination_city, start_date, end_date, adults)

        # Step 4: Generate itinerary
        print("Step 4: Creating your personalized travel itinerary...")
        itinerary = self.itinerary_agent.process_request(
            destination_city, start_date, end_date, flights, weather, hotels
        )

        print("\nYour travel itinerary is ready!")
        return itinerary

    def interactive_planning(self):
        """Interactive mode for planning a trip with user input"""
        print("\nWelcome to the AI Travel Assistant!\n")

        # Get origin
        origin = input("Please enter your departure airport code (e.g., JFK): ").strip().upper()

        # Get destination
        destination = input("Please enter your destination airport code (e.g., CDG): ").strip().upper()

        # Get travel dates
        start_date = input("Please enter your departure date (YYYY-MM-DD): ").strip()
        end_date = input("Please enter your return date (YYYY-MM-DD): ").strip()

        # Get number of travelers
        try:
            adults = int(input("Please enter the number of travelers: ").strip())
        except ValueError:
            print("Using default value of 1 traveler.")
            adults = 1

        # Generate and print the itinerary
        itinerary = self.plan_trip(origin, destination, start_date, end_date, adults)
        print("\n" + itinerary)

        return itinerary


# Main execution block
if __name__ == "__main__":
    print("Travel Assistant Application")
    print("===========================")
    print("\nInitializing travel assistant with API connections...")

    # Create travel assistant and run interactive planning
    try:
        travel_assistant = TravelAssistantOrchestrator()
        travel_assistant.interactive_planning()
    except KeyboardInterrupt:
        print("\n\nTravel planning canceled by user. Goodbye!")
    except Exception as e:
        print(f"\nAn error occurred: {str(e)}")
        print("Please check your internet connection and API keys, then try again.")

Travel Assistant Application

Initializing travel assistant with API connections...

Welcome to the AI Travel Assistant!

Please enter your departure airport code (e.g., JFK): JFK
Please enter your destination airport code (e.g., CDG): CDG
Please enter your departure date (YYYY-MM-DD): 2025-04-25
Please enter your return date (YYYY-MM-DD): 2025-04-28
Please enter the number of travelers: 2

==================== TRAVEL ASSISTANT ====================
Planning trip from JFK to CDG
Travel dates: 2025-04-25 to 2025-04-28
Number of travelers: 2

Step 1: Retrieving flight options...
[Flight Agent] Authenticating with Amadeus API...
[Flight Agent] Authentication successful
[Flight Agent] Searching flights from JFK to CDG on 2025-04-25
[Flight Agent] Found 5 flight options
Step 2: Retrieving weather forecast for Paris...
[Weather Agent] Retrieving weather forecast for Paris from 2025-04-25 to 2025-04-28
[Weather Agent] Retrieved weather data for 4 days
Step 3: Finding accommodation in Paris...
